## Get Results from old simulation
To run a new simulation, execute the `run_sim.py` script.
If the command below fails, you probably need to run a new simulation to generate data.

In [ ]:
# Specific imports for cellular_raza
# The package is named cr_autophagy
# We want to reload the package when some of the behind-the scenes python functions change
# This is what the importlib statements are for
import importlib
import cr_pool_model as crp
importlib.reload(crp)

# Imports of general-purpose python libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

output_path = crp.get_last_output_path()
# output_path = Path("out/pool_model/2023-12-07-T01-53-41")
domain, initial_cells, meta_params = crp.get_simulation_settings(output_path)
max_iter = max(crp.get_all_iterations(output_path))

In [ ]:
# Cellular Properties and their names in dataframe
iter_0_particles = crp.get_elements_at_iter(output_path, 0)
print("Name".ljust(62), "Type")
print("⎯"*80)
for ty, col in zip(iter_0_particles.dtypes, iter_0_particles.columns):
    print("{}".format(col).ljust(62), "{}".format(ty))

In [ ]:
# Voxel Properties and their names in dataframe
iter_0_voxels = crp.get_elements_at_iter(output_path, 0, element_path="voxel_storage")
print("Name".ljust(62), "Type")
print("⎯"*80)
for ty, col in zip(iter_0_voxels.dtypes, iter_0_voxels.columns):
    print("{}".format(col).ljust(62), "{}".format(ty))

In [ ]:
# Efficiently Gather Data
all_iterations = crp.get_all_iterations(output_path)

# Define units for time
UNIT_HOUR = 60
UNIT_DAY = 24*UNIT_HOUR

def analyze_voxel_data(output_path, iteration):
    # Calculate total extracellular concentrations
    entry = crp.get_elements_at_iter(output_path, iteration, element_path="voxel_storage")
    mi = np.array([x for x in entry["element.voxel.min"]])
    ma = np.array([x for x in entry["element.voxel.max"]])
    diff = np.abs(ma-mi)
    conc = np.array([c for c in entry["element.voxel.extracellular_concentrations"]])
    volume = np.prod(diff, axis=1)
    conc_total = np.sum(volume[:,None] * conc, axis=0)

    return {
        "iteration": iteration,
        "time": iteration*meta_params.dt/UNIT_DAY,
        "nutrients_conc_total": conc_total[0],
        "inhib_total": conc_total[1],
    }


def _analyze_voxel_data_helper(args):
    return analyze_voxel_data(*args)


def analyze_cell_data(output_path, iteration):
    entry = crp.get_elements_at_iter(output_path, iteration)

    # Calculate number of cells for species
    bacteria_1 = entry[entry["element.cell.cellular_reactions.species"]=="S1"]
    bacteria_2 = entry[entry["element.cell.cellular_reactions.species"]=="S2"]
    bacteria_count_1 = len(bacteria_1)
    bacteria_count_2 = len(bacteria_2)

    # Calculate the number of bacteria in lag phase
    bacteria_in_lag_phase_1 = len(bacteria_1[bacteria_1["element.cell.cellular_reactions.lag_phase_active"]==True])
    bacteria_in_lag_phase_2 = len(bacteria_2[bacteria_2["element.cell.cellular_reactions.lag_phase_active"]==True])

    # Calculate total intracellular concentrations
    volume = np.array([x for x in entry["element.cell.cellular_reactions.cell_volume"]])

    volume_1 = np.sum(volume[entry["element.cell.cellular_reactions.species"]=="S1"])
    volume_2 = np.sum(volume[entry["element.cell.cellular_reactions.species"]=="S2"])
    total_cell_volume = np.sum(volume)

    # Calculate the entropy at this step
    entropy = crp.calculate_entropy(output_path, iteration)

    # Return combined results
    return {
        "iteration": iteration,
        "time": iteration*meta_params.dt/UNIT_DAY,
        "bacteria_count_1": bacteria_count_1,
        "bacteria_count_2": bacteria_count_2,
        "bacteria_count_total": bacteria_count_1 + bacteria_count_2,
        "bacteria_in_lag_phase_1": bacteria_in_lag_phase_1,
        "bacteria_in_lag_phase_2": bacteria_in_lag_phase_2,
        "bacteria_in_lag_phase_total": bacteria_in_lag_phase_1 + bacteria_in_lag_phase_2,
        "bacteria_volume_1": volume_1,
        "bacteria_volume_2": volume_2,
        "bacteria_volume_total": total_cell_volume,
        "entropy": entropy,
    }


def _analyze_cell_data_helper(args):
    return analyze_cell_data(*args)


# Construct pool to process results in parallel
pool = crp.mp.Pool(crp.mp.cpu_count()-4)

# Gather args and use pool to get already pre-processed results
args = [(output_path, iteration) for iteration in crp.get_all_iterations(output_path)]
data_cells = crp.pd.DataFrame(crp.tqdm.tqdm(pool.imap_unordered(_analyze_cell_data_helper, args), total=len(args))).sort_values("iteration").reset_index(drop=True)
data_voxels = crp.pd.DataFrame(crp.tqdm.tqdm(pool.imap_unordered(_analyze_voxel_data_helper, args), total=len(args))).sort_values("iteration").reset_index(drop=True)

# Close the pool to free any remaining resources
pool.close()
pool.join()

In [ ]:
# Growth Curve
fig, ax1 = plt.subplots()

data_cells.plot(x="time", y="bacteria_count_1", ax=ax1, label="Species 1", color="#252B3370")
data_cells.plot(x="time", y="bacteria_count_2", ax=ax1, label="Species 2", color="#D1402770")
data_cells.plot(x="time", y="bacteria_count_total", ax=ax1, label="Combined", color="#7E57A570")
ax1.set_ylabel("Bacteria Count")
ax1.set_xlabel("Time [days]")

ax2 = ax1.twinx()
data_cells.plot(x="time", y="bacteria_volume_1", ax=ax2, label="Species 1", color="#252B33")
data_cells.plot(x="time", y="bacteria_volume_2", ax=ax2, label="Species 2", color="#D14027")
data_cells.plot(x="time", y="bacteria_volume_total", ax=ax2, label="Combined", color="#7E57A5")
ax2.set_ylabel("Combined Bacteria Volume")

ax1.legend()
fig.tight_layout()
fig.savefig(f"{output_path}/cell_growth.png")
fig.savefig(f"{output_path}/cell_growth.pdf")
plt.show(fig)

In [ ]:
# Nutrients
fig, ax = plt.subplots()
data_voxels.plot(x="time", y="nutrients_conc_total", ax=ax, label="Extracellular Nutrients", color='#808080')
ax.set_ylabel("Total Nutrients")
ax.set_xlabel("Time [days]")
plt.show(fig)

In [ ]:
# TODO Compare Spatial Model to ODE model

import pool_paper_spatial as pool_ode
importlib.reload(pool_ode)

def abm_to_ode(
        domain: crp.Domain,
        initial_cells: list[crp.BacteriaTemplate],
        meta_params: crp.MetaParams
    ) -> pool_ode.ODEModel:
    kwargs = {}

    # Define Initial Values
    mask = np.array([c.cellular_reactions.species=="S1" for c in initial_cells])
    volumes = np.array([c.cellular_reactions.cell_volume for c in initial_cells])
    bacteria_volume_1 = np.sum(volumes[mask])
    bacteria_volume_2 = np.sum(volumes[mask==False])

    kwargs["initial_total_volume_lag_phase_1"] = bacteria_volume_1
    kwargs["initial_total_volume_lag_phase_2"] = bacteria_volume_2

    # Define OdeParameters

    # lag phase
    kwargs["lag_phase_transition_rate_1"] = np.average([
        c.cycle.lag_phase_transition_rate_1 for c in initial_cells
    ])
    kwargs["lag_phase_transition_rate_2"] = np.average([
        c.cycle.lag_phase_transition_rate_2 for c in initial_cells
    ])

    # Inhibitor Production rate
    abm_cell_volume_max = np.average([
        c.cycle.volume_division_threshold for c in initial_cells
    ])
    abm_cell_volume_min = abm_cell_volume_max / 2.0
    abm_cell_volume_average = 0.5*(abm_cell_volume_max + abm_cell_volume_min)
    abm_domain_volume = domain.size**2
    abm_inhibition_production_rate = np.average([
        c.cellular_reactions.inhibition_production_rate for c in initial_cells
    ])
    kwargs["inhibitor_production"] = abm_inhibition_production_rate

    abm_inhibition_coefficient = np.average([
        c.cellular_reactions.inhibition_coefficient for c in initial_cells
    ])

    kwargs["inhibition_constant"] = abm_inhibition_coefficient / abm_domain_volume
    abm_initial_food = abm_domain_volume * domain.initial_concentrations[0]
    abm_food_to_volume = np.average([
        c.cellular_reactions.food_to_volume_conversion for c in initial_cells
    ])

    # total cell volume instead of number of cells
    kwargs["total_cell_volume"] = abm_initial_food*abm_food_to_volume
    
    uptake_rates = np.array([
        c.cellular_reactions.uptake_rate for c in initial_cells
    ])
    abm_uptake_rate_1 = np.average(uptake_rates[mask])
    abm_uptake_rate_2 = np.average(uptake_rates[mask==False])

    kwargs["production_rate_1"] = abm_uptake_rate_1\
        * abm_food_to_volume\
        * domain.initial_concentrations[0]
    kwargs["production_rate_2"] = abm_uptake_rate_2\
        * abm_food_to_volume\
        * domain.initial_concentrations[0]

    # Meta Parameters
    kwargs["time_start"]            = meta_params.t_start
    kwargs["time_dt"]               = meta_params.dt
    kwargs["time_steps"]            = meta_params.n_times
    kwargs["time_save_interval"]    = meta_params.save_interval

    # Define OdeMetaParameters
    ode_model = pool_ode.ODEModel(
        **kwargs
    )
    return ode_model


def calculate_difference(output_path):
    settings = crp.get_simulation_settings(output_path)
    
    ode_model = abm_to_ode(*settings)
    res = ode_model.solve_ode_raw()
    obs_ode = pool_ode.observable_2pool_2species(res)

    obs_abm = np.array([
        data_cells[["bacteria_volume_1", "bacteria_volume_2", "bacteria_volume_total"]]
    ]).reshape((obs_ode.T.shape)).T

    variance = np.average(((obs_ode - obs_abm)/ode_model.total_cell_volume)**2)**0.5
    return variance


# diff = calculate_difference(output_path)

settings = crp.get_simulation_settings(output_path)

model = abm_to_ode(*settings)
print(model)
res = model.solve_ode_raw()
obs = pool_ode.observable_2pool_2species(res)

fig, ax = plt.subplots()

ax.plot(res.t/UNIT_DAY, obs[0], color="#252B33A0", label="Species A (ODE)")
ax.plot(res.t/UNIT_DAY, obs[1], color="#D14027A0", label="Species B (ODE)")
ax.plot(res.t/UNIT_DAY, obs[2], color="#7E57A5A0", label="Combined (ODE)")

data_cells.plot(x="time", y="bacteria_volume_1", ax=ax, label="Species A (ABM)", color="#252B33", linestyle="--")
data_cells.plot(x="time", y="bacteria_volume_2", ax=ax, label="Species B (ABM)", color="#D14027", linestyle="--")
data_cells.plot(x="time", y="bacteria_volume_total", ax=ax, label="Combined (ABM)", color="#7E57A5", linestyle="--")

ax.set_xlabel("Time [days]")
ax.set_ylabel("Combined Bacteria Volume")
# ax.set_yscale('log')
ax.legend()
fig.savefig(output_path / "abm_ode_comparison.png")
fig.savefig(output_path / "abm_ode_comparison.pdf")
fig.savefig(output_path / "abm_ode_comparison.eps")
plt.show(fig)

In [ ]:
# Lag Phase
fig, ax = plt.subplots()
ax.set_title("Cells in Lag-Phase")

ode_lag_phase_1 = res.y[0]
ode_lag_phase_2 = res.y[2]
ode_lag_phase_combined = res.y[0] + res.y[2]

ax.plot(res.t/UNIT_DAY, ode_lag_phase_1/(np.pi*1.5**2), label="Species 1 (ODE)", color="#252B33", linestyle="--")
ax.plot(res.t/UNIT_DAY, ode_lag_phase_2/(np.pi*1.5**2), label="Species 2 (ODE)", color="#D14027", linestyle="--")
ax.plot(res.t/UNIT_DAY, ode_lag_phase_combined/(np.pi*1.5**2), label="Combined (ODE)", color="#7E57A5", linestyle="--")

data_cells.plot(x="time", y="bacteria_in_lag_phase_1", ax=ax, label="Species 1 (ABM)", color="#252B33")
data_cells.plot(x="time", y="bacteria_in_lag_phase_2", ax=ax, label="Species 2 (ABM)", color="#D14027")
data_cells.plot(x="time", y="bacteria_in_lag_phase_total", ax=ax, label="Combined (ABM)", color="#7E57A5")

ax.legend()
ax.set_xlabel("Time [days]")
ax.set_ylabel("Bacteria Volume")

fig.tight_layout()
fig.savefig(f"{output_path}/lag_phase.png")
fig.savefig(f"{output_path}/lag_phase.pdf")
plt.show(fig)

In [ ]:
# Entropy
entropies = np.array([x for x in data_cells["entropy"]])

fig, ax = plt.subplots()
ax.set_title("Spatial Density Entropy")
ax.plot(data_cells["time"], entropies[:,0], label="Species 1", color='#D06062')
ax.plot(data_cells["time"], entropies[:,1], label="Species 2", color='#4E89B1')
ax.legend()
ax.set_xlabel("Time [days]")
ax.set_ylabel("Shannon Entropy")
fig.tight_layout()
fig.savefig(output_path / "entropy.png")
fig.savefig(output_path / "entropy.pdf")
plt.show(fig)

In [ ]:
def _save_psychic_image(output_path, iteration):
    import scipy as sp
    data = crp.get_elements_at_iter(output_path, iteration)
    domain, _, _ = crp.get_simulation_settings(output_path)
    fig, ax = crp._create_base_canvas(domain)

    data1 = data[data["element.cell.cellular_reactions.species"]=="S1"]
    data2 = data[data["element.cell.cellular_reactions.species"]!="S1"]

    Z1 = crp.calculate_spatial_density(data1, domain, weights=True)
    Z2 = crp.calculate_spatial_density(data2, domain, weights=True)

    Z = Z1/(Z1+Z2)

    ax.imshow(np.rot90(Z), cmap=plt.cm.viridis,
        extent=[0.0, domain.size, 0.0, domain.size])

    df_cells = crp.get_elements_at_iter(output_path, iteration)

    crp._plot_bacteria(df_cells, ax)

    ax.set_xlim([0.0, domain.size])
    ax.set_ylim([0.0, domain.size])

    # ax.set_title(f"Entropy {sp.stats.entropy(Z.reshape(-1)):4.2f}")
    crp._plot_labels(fig, ax, n_bacteria_1=len(data1), n_bacteria_2=len(data2))

    save_path = crp._determine_image_save_path(output_path, iteration)
    fig.savefig(save_path, bbox_inches='tight', pad_inches=0)

def _save_psychic_image_helper(args):
    _save_psychic_image(*args)

# _save_psychic_image(output_path, max_iter)

# import multiprocessing as mp
# import tqdm
# args = [(output_path, iteration) for iteration in crp.get_all_iterations(output_path)]
# _ = list(tqdm.tqdm(mp.Pool().imap_unordered(_save_psychic_image_helper, args), total=len(args)))

In [ ]:
# Save snapshots and generate Movie
import os
crp.save_all_snapshots(output_path, threads=-1)

# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -v quiet -stats -y -r 30 -f image2 -pattern_type glob -i '{output_path}/images/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/movie_4.mp4"
os.system(bashcmd)

In [ ]:
crp.save_snapshot(output_path,  6_000, formats=["png", "pdf", "eps"])
crp.save_snapshot(output_path, 12_000, formats=["png", "pdf", "eps"])
crp.save_snapshot(output_path, 18_000, formats=["png", "pdf", "eps"])
crp.save_snapshot(output_path, 24_000, formats=["png", "pdf", "eps"])